In [7]:
import numpy as np
import skimage as sk
import skimage.io as skio
from sklearn.preprocessing import StandardScaler
from skimage.filters import roberts, sobel, scharr, prewitt
from skimage.feature import canny

# Shifts image by a given x,y displacement
def shift(image, displacement):
    return np.roll(np.roll(image, displacement[0], axis = 0), displacement[1], axis = 1)

# aligns two images together using SSD
def align(A, B):
    displacement = [0,0]
    ssd = np.sum(np.sum(np.square(A - B)))
    
    edge_A = roberts(A)
    edge_B = roberts(B)
    
    for i in range(-15, 15):
        modx_A = np.roll(A, i, axis = 0)
        for j in range(-15, 15):
            modxy_A = np.roll(modx_A, j, axis = 1)
            mod_ssd = np.sum(np.sum(np.square(modxy_A - B)))
            if (mod_ssd < ssd):
                ssd = mod_ssd
                displacement = [i, j]
    
    return displacement

# pyramid imaging to align two high resolution images.
def pyramid_align(A, B, level):
    if A.shape[0] <= 400 or B.shape[0] <= 400 or level == 5:
        return align(A,B)
    else:
        displacement = pyramid_align(sk.transform.rescale(A, .5), sk.transform.rescale(B, .5), level + 1)
        scaled_displacement = [displacement[0] * 2, displacement[1] * 2]
        rel_A = shift(A, scaled_displacement)
        
        x_reduction = rel_A.shape[0]//2 - 100
        y_reduction = rel_A.shape[1]//2 - 100
        displacement = align(rel_A[x_reduction: - x_reduction, y_reduction: - y_reduction], B[x_reduction: - x_reduction, y_reduction: - y_reduction])
        result = [scaled_displacement[0] + displacement[0], scaled_displacement[1] + displacement[1]]
        return result

# method initializer for pyramid aligning
def pyramid_method(A,B):
    croppedA = A[800:-800, 800:-800]
    croppedB = B[800:-800, 800:-800]
    
    return pyramid_align(A, B, 1)

def colorize_jpeg(filename):
    # name of the input file
    imname = filename + ".jpg"

    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)

    # compute the height of each part (just 1/3 of total)
    height = int(np.floor(im.shape[0] / 3.0))
    
    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]
    
    ag_displacement = align(g[20:-20, 20:-20], b[20:-20, 20:-20])
    ar_displacement = align(r[20:-20, 20:-20], b[20:-20, 20:-20])

    ag = shift(g, ag_displacement)
    ar = shift(r, ar_displacement)
    
    print(filename)
    print("green: " + str(ag_displacement[0]) + ", " + str(ag_displacement[1]))
    print("red: " + str(ar_displacement[0]) + ", " + str(ar_displacement[1]))

    # create a color image
    im_out = np.dstack([ar, ag, b])

    # save the image
    fname = 'out_' + filename + '.jpeg'
    skio.imsave(fname, im_out)
    
def colorize_tif(filename):
    # name of the input file
    imname = filename + ".tif"

    # read in the image
    im = skio.imread(imname)

    # convert to double (might want to do this later on to save memory)    
    im = sk.img_as_float(im)

    # compute the height of each part (just 1/3 of total)
    height = int(np.floor(im.shape[0] / 3.0))
    
    # separate color channels
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]
    
    ag_displacement = pyramid_method(g,b)
    ar_displacement = pyramid_method(r,b)

    ag = shift(g, ag_displacement)
    ar = shift(r, ar_displacement)
    
    print(filename)
    print("green: " + str(ag_displacement[0]) + ", " + str(ag_displacement[1]))
    print("red: " + str(ar_displacement[0]) + ", " + str(ar_displacement[1]))

    # create a color image
    im_out = np.dstack([ar, ag, b])

    # save the image
    fname = 'out_' + filename + '.jpeg'
    skio.imsave(fname, im_out)
    
def main():
    jpg_image_names = ['cathedral', 'monastery', 'nativity', 'settlers']
    tif_image_names = ['emir', 'harvesters', 'icon', 'lady', 'self_portrait', 'three_generations', 'train', 'turkmen', 'village']
    
    for filename in jpg_image_names:
        colorize_jpeg(filename)
        
    for filename in tif_image_names:
        colorize_tif(filename)
    
main()

cathedral
green: 5, 2
red: 12, 3


c:\users\kayce\miniconda3\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


monastery
green: -3, 2
red: 3, 2
nativity
green: 3, 1
red: 7, 1
settlers
green: 7, 0
red: 14, -1


c:\users\kayce\miniconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


emir
green: 48, 23
red: 68, 40
harvesters
green: 60, 18
red: 124, 17
icon
green: 40, 18
red: 89, 24
lady
green: 48, 8
red: 115, 8
self_portrait
green: 77, 29
red: 175, 37
three_generations
green: 49, 17
red: 108, 13
train
green: 42, 6
red: 85, 32
turkmen
green: 56, 22
red: 117, 30
village
green: 129, 83
red: 136, 23
